
we are going to compute the probablity for the following problem :
we have 700 random variables and we have the probablity for each random variable and we want to 
compute the probabilty for the S > = 800 such as \begin{equation*} S = \sum_{i=1}^{700} x_i \end{equation*} 
it's a toy problem for the SBU Statics class of 2018 :D made by yours truly Pooria Poorsarvi <br>
refrences :<br>
[wikipedia (Persian)](https://fa.wikipedia.org/wiki/%D8%AA%D9%88%D8%B2%DB%8C%D8%B9_%DA%86%D9%86%D8%AF%D8%AC%D9%85%D9%84%D9%87%E2%80%8C%D8%A7%DB%8C)
<br>
[wikipedia (English)](https://en.wikipedia.org/wiki/Multinomial_distribution)


In [2]:
import numpy as np
from mpmath import mpf, mpc, mp
from itertools import permutations
class RandomVariable:
    def __init__(self,_domain,_p,withInit=False):
        self.domain = _domain
        self.p = [mpf(i) for i in _p]
        self.value = None
        if(withInit):
            self.value = np.random.choice(self.domain,p=self.p)            
    def assignValue(self):
        self.value = np.random.choice(self.domain,p=self.p)

In [3]:
rv = RandomVariable([-1,0,2],[1/4,1/2,1/4])

In [4]:
for i in range(20) :
    rv.assignValue()
    print(rv.value)

-1
0
0
-1
2
-1
2
0
-1
-1
0
0
-1
-1
0
-1
0
-1
-1
-1


In [5]:
class Computations:
    def __init__(self, _upBound=701):
        self.facts = [1]
        self.upBound = _upBound
        for i in range(1,_upBound):
            self.facts.append((self.facts[-1]*i))
    
    def choice(self, n, x):
        if(n<x):
            return 0
        
        if(n>=self.upBound):
            raise ArithmeticError("the required factorial was higher then the _upBound")
        
        return (self.facts[n]/(self.facts[x]*self.facts[n-x]))
    
    

In [6]:
c = Computations()

first we will try to use Diophantine equation and Multinomial distribution :D

In [7]:
class Population:
    def __init__(self, _domain=None, _p=None, _randomVars=None,maxNum=701):
        '''
        you either give the domain and the probablity or you give the random variables :D
        '''
        self.domain = _domain
        
        if(_p is None):
            self.p = _p
        else :
            self.p = [mpf(i) for i in _p]
            
        
        self.randomVars = _randomVars
        self.randomVarsInfo = None
        
        if(self.domain is None):
            
            if(type(self.randomVars) is list):
                
                if not (self.randomVars[0] is None):
                    
                    if not (self.randomVars[0][0] is None):
                        
                        self.domain = self.randomVars[0][0].domain
                        self.p = self.randomVars[0][0].p
                        self.makeCheck()
                        
                        
                        
        self.computer = Computations(_upBound=maxNum+1)
        
        if (self.domain is None):
            raise ArithmeticError("we can't use empty sets sorry :(")
        
        if not (sum(self.p) == 1.0):
            raise ArithmeticError("the sum of probabilities should be equal to one")
        
        if not (len(self.domain) is len(self.p)):
            raise ArithmeticError("you should provide probablity for each class")
    
    
    
    def makeCheck(self):
        
        '''
        adds the info of each population to the randomVarsInfo
        '''
        
        self.randomVarsInfo = []
        
        
        for i in self.randomVars:
            temp = dict(zip(self.domain, [0 for z in self.domain]))
            for j in i :
                temp[j.value]+=1
            self.randomVarsInfo.append(temp)
            
            
                
    
    def computeProb(self, howMuch):
        '''
        this parts uses Diophantine equation
        how much contains how much we should use of each class , for example if domain is [0,1,2]
        and howMuch is [10,2,4] then we have 10x0 and 2x1 and 4x2
        take in considaration that the sum of the howMuch var is equal to the number of RandomVariables
        '''
        if not (len(self.domain) is len(howMuch)):
            raise ArithmeticError("the provided array should have the same number of ints as the number of classes")
        overAllRemaining = sum(howMuch)
        res = 1
        counter = 0
        stringRes = ""
        for i in howMuch:
            res *= self.computer.choice(overAllRemaining, i)*(self.p[counter]**i)
            stringRes = stringRes + "c( "+str(overAllRemaining)+" , "+str(i)+" ) * ( "+str(self.p[counter])+" ) ^ "+str(i)
            overAllRemaining -= i
            counter += 1
            if not (counter is len(howMuch)):
                stringRes += " * "
        return res,stringRes
    
    
    
    def generatePopulations(self, howManyPopulations, howManyPerEach):
        
        '''
        genarate populations :D
        '''
        self.randomVars = []
        
        for i in range(howManyPopulations):
            temp = []
            for j in range(howManyPerEach):
                temp.append(RandomVariable(self.domain, self.p, withInit=True))
            self.randomVars.append(temp)
        self.makeCheck()
    
    def printGenerations(self):
        '''
        generates the populations and stores theire info
        '''
        for i in self.randomVars:
            temp = []
            
            for j in i :
                temp.append(str(j.value))
            
            print(temp)
            
        for i in self.randomVarsInfo:
            temp = []
            for j in i :
                temp.append(str(j)+" "+str(i[j]))
            print(temp)
    
    def getProbGenerations(self, howMuch):
        '''
        generates a probability based on what it has seen
        '''
        whatWeWant = dict(zip(self.domain, howMuch))
        res = 0
        for i in self.randomVarsInfo:
            if(i==whatWeWant):
                res += 1
        return res / len(self.randomVarsInfo)

In [8]:
_domain = [-1, 0, 2]
_p = [1/4, 1/8, 5/8]

In [9]:
population = Population(_domain = _domain, _p = _p)

In [10]:
res = 0
sits = []
howMany = 700
sumExcpected = -700
setSeen = set()
for i in range(howMany+1):
    for j in range(howMany+1-i):
        z = howMany-i-j
        perm = set(permutations([i,j,z]))
        for per in perm:
            resSum = 0
            boolean = False
            if(per in setSeen):
                continue
            setSeen.add(per)
            for d in range(len(per)):
                resSum += _domain[d]*per[d]
            if(resSum>=sumExcpected):
                boolean = True
                resTemp,resStringTemp = population.computeProb(per)
                res+=resTemp
                sits.append(per)

In [11]:
res

mpf('0.99999999999999001')

In [12]:
res = 0
sits = []
howMany = 700
sumExcpected = 800
setSeen = set()
for i in range(howMany+1):
    for j in range(howMany+1-i):
        z = howMany-i-j
        perm = set(permutations([i,j,z]))
        for per in perm:
            resSum = 0
            boolean = False
            if(per in setSeen):
                continue
            setSeen.add(per)
            for d in range(len(per)):
                resSum += _domain[d]*per[d]
            if(resSum>=sumExcpected):
                boolean = True
                resTemp,resStringTemp = population.computeProb(per)
                res+=resTemp
                sits.append(per)

In [13]:
res

mpf('0.0020233744094019898')

from here we will use simulation :D

In [14]:
resSimul = 0 
population.generatePopulations(1000,700)

In [15]:
for i in sits :
    resSimul+=population.getProbGenerations(i)

In [16]:
resSimul

0.003

In [17]:
resSimul = 0 
population.generatePopulations(100,700)

In [18]:
for i in sits :
    resSimul+=population.getProbGenerations(i)

In [19]:
resSimul

0.0

from here we will see how to use the class

In [20]:
a = Population([-1 , 0 , 2],[1/4,1/2,1/4])

In [21]:
res,strRes = a.computeProb([700,0,0])

In [22]:
a.generatePopulations(10,1)

In [23]:
a.printGenerations()

['2']
['0']
['-1']
['0']
['0']
['0']
['2']
['-1']
['0']
['-1']
['-1 0', '0 0', '2 1']
['-1 0', '0 1', '2 0']
['-1 1', '0 0', '2 0']
['-1 0', '0 1', '2 0']
['-1 0', '0 1', '2 0']
['-1 0', '0 1', '2 0']
['-1 0', '0 0', '2 1']
['-1 1', '0 0', '2 0']
['-1 0', '0 1', '2 0']
['-1 1', '0 0', '2 0']


In [24]:
a.getProbGenerations([1,0,0])

0.3

In [25]:
res, strRes = a.computeProb([1,0,1])

In [26]:
print(res)
print(strRes)

0.125
c( 2 , 1 ) * ( 0.25 ) ^ 1 * c( 1 , 0 ) * ( 0.5 ) ^ 0 * c( 1 , 1 ) * ( 0.25 ) ^ 1


In [49]:
def getFunc(population1, howMany1 = 700, sumExcpected1 = -700):
    res1 = 0
    sits1 = []
    setSeen1 = set()
    for i in range(howMany1+1):
        for j in range(howMany1+1-i):
            z = howMany1-i-j
            perm = set(permutations([i,j,z]))
            for per in perm:
                resSum = 0
                boolean = False
                if(per in setSeen1):
                    continue
                setSeen1.add(per)
                for d in range(len(per)):
                    resSum += _domain[d]*per[d]
                if(resSum==sumExcpected1):
                    boolean = True
                    resTemp,resStringTemp = population.computeProb(per)
                    res1+=resTemp
                    sits1.append(per)
    return res1

In [52]:
results = dict()

In [53]:
for i in range(-700,1401):
    if(i%100==0):
        print(i)
    res = getFunc(population,sumExcpected1=i)
    results.update({i:res})

-700
-600
-500
-400
-300
-200
-100
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400


In [55]:
listAnswers = []
for i in range(-700,1401):
    listAnswers.append([i,results[i]])
    

In [72]:
import pandas as pd

In [81]:
listAnswers = np.array(listAnswers)

In [82]:
listAnswers.shape

(2101, 2)

In [88]:
listAnswers[:,:1]

array([[-700],
       [-699],
       [-698],
       ...,
       [1398],
       [1399],
       [1400]], dtype=object)

In [89]:
listAnswers[:,1:]

array([[mpf('3.6141491434385841e-422')],
       [mpf('1.2649522002035044e-419')],
       [mpf('2.210503969855624e-417')],
       ...,
       [mpf('1.8286903643424991e-141')],
       [0],
       [mpf('1.306207403101785e-143')]], dtype=object)

In [90]:
df = pd.DataFrame(listAnswers[:,1:],index=listAnswers[:,:1])

In [91]:
df.to_csv('savedForAmar.csv',header=False)

In [92]:
df

,0
"(-700,)",3.61414914343858e-422
"(-699,)",1.2649522002035e-419
"(-698,)",2.21050396985562e-417
"(-697,)",2.57218542769881e-415
"(-696,)",2.24267601305023e-413
"(-695,)",1.56322132374734e-411
"(-694,)",9.07613130354453e-410
"(-693,)",4.51595597805476e-408
"(-692,)",1.96620764759888e-406
"(-691,)",7.6117688978513e-405
